In [1]:
import numpy as np
from utils import *
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from scipy.misc import logsumexp
import random
import csv

In [2]:
# superclass of modules
class Module:
    """
    Module is a super class. It could be a single layer, or a multilayer perceptron.
    """
    
    def __init__(self):
        self.train = True
        return
    
    def forward(self, _input):
        """
        h = f(z); z is the input, and h is the output.
        
        Inputs:
        _input: z
        
        Returns:
        output h
        """
        pass
    
    def backward(self, _input, _gradOutput):
        """
        Compute:
        gradient w.r.t. _input
        gradient w.r.t. trainable parameters
        
        Inputs:
        _input: z
        _gradOutput: dL/dh
        
        Returns:
        gradInput: dL/dz
        """
        pass
        
    def parameters(self):
        """
        Return the value of trainable parameters and its corresponding gradient (Used for grandient descent)
        
        Returns:
        params, gradParams
        """
        pass
    
    def training(self):
        """
        Turn the module into training mode.(Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = True
        
    def evaluate(self):
        """
        Turn the module into evaluate mode.(Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = False
        

In [3]:
class Sequential(Module):
    """
    Sequential provides a way to plug layers together in a feed-forward manner.
    """
    def __init__(self):
        Module.__init__(self)
        self.layers = [] # layers contain all the layers in order
    
    def add(self, layer):
        self.layers.append(layer) # Add another layer at the end
    
    def size(self):
        return len(self.layers) # How many layers.
    
    def forward(self, _input):
        """
        Feed forward through all the layers, and return the output of the last layer
        """
        # self._inputs saves the input of each layer
        # self._inputs[i] is the input of i-th layer
        self._inputs = [_input]
        
        # YOUR CODE HERE
        for i in range(self.size()):
            current_input = self._inputs[i]
            layer = self.layers[i]
            self._inputs.append(layer.forward(current_input))
        
        # The last element of self._inputs is the output of last layer
        self._output = self._inputs[-1]
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        Backpropogate through all the layers using chain rule.
        """
        # self._gradInputs[i] is the gradient of loss w.r.t. the input of i-th layer
        self._gradInputs = [None] * (self.size() + 1)
        self._gradInputs[self.size()] = _gradOutput

        # YOUR CODE HERE
        for i in range(self.size(), 0, -1):
#             print("{} layer".format(i))
            current_input = self._inputs[i - 1]
#             print("length of gradinput {}".format(len(self._gradInputs)))
            current_gradoutput = self._gradInputs[i]
            layer = self.layers[i - 1]
            gradInput = layer.backward(current_input, current_gradoutput)
            self._gradInputs[i - 1] = gradInput
        
        self._gradInput = self._gradInputs[0]
        return self._gradInput
    
    def parameters(self):
        """
        Return trainable parameters and its corresponding gradient in a nested list
        """
        params = []
        gradParams = []
        for m in self.layers:
            _p, _g = m.parameters()
            if _p is not None:
                params.append(_p)
                gradParams.append(_g)
        return params, gradParams

    def training(self):
        """
        Turn all the layers into training mode
        """
        Module.training(self)
        for m in self.layers:
            m.training()
    
    def evaluate(self):
        """
        Turn all the layers into evaluate mode
        """
        Module.evaluate(self)
        for m in self.layers:
            m.evaluate()
        

In [4]:
class FullyConnected(Module):
    """
    Fully connected layer
    """
    def __init__(self, inputSize, outputSize):
        Module.__init__(self)
        # Initalization
        stdv = 1./np.sqrt(inputSize)
        
        self.weight = np.random.uniform(-stdv, stdv, (outputSize, inputSize))
        self.gradWeight = np.ndarray((outputSize, inputSize))
        self.bias = np.random.uniform(-stdv, stdv, outputSize)
        self.gradBias = np.ndarray(outputSize)
        
    def forward(self, _input):
        """
        output = W * input + b
        
        _input:
        N x inputSize matrix
        
        """
        # YOUR CODE HERE
        N = _input.shape[0]
#         outputSize = self.bias.shape[1]
        self._output = np.dot(self.weight, _input.T).T + self.bias
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        _input:
        N x inputSize matrix
        _gradOutputSize:
        N x outputSize matrix
        """
#         self.gradWeight = # YOUR CODE HERE
#         self.gradBias = # YOUR CODE HERE
        
#         self._gradInput = # YOUR CODE HERE

        N = _input.shape[0]
        inputSize = _input.shape[1]
        outputSize = _gradOutput.shape[1]
        gradWeight_ = [np.dot(_gradOutput[i].reshape([outputSize, 1]), _input[i].reshape([1, inputSize]))
                          for i in range(N)]
        # sum or mean?
        self.gradWeight = np.sum(gradWeight_, axis = 0)
        self.gradBias = np.sum(_gradOutput, axis = 0)
        
        self._gradInput = np.dot(self.weight.T, _gradOutput.T).T
        return self._gradInput
        
    def parameters(self):
        """
        Return weight and bias and their g
        """
        return [self.weight, self.bias], [self.gradWeight, self.gradBias]

In [5]:
class ReLU(Module):
    """
    ReLU activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
        return
    
    def forward(self, _input):
        """
        output = max(0, input)
        
        _input:
        N x d matrix
        """
        #YOUR CODE HERE
        self._output = np.maximum(_input, np.zeros_like(_input))
        return self._output
    
    def backward(self, _input, _gradOutput):
        """
        gradInput = gradOutput * mask
        mask = _input > 0
        
        _input:
        N x d matrix
        
        _gradOutput:
        N x d matrix
        """
        # YOUR CODE HERE
        mask = (_input > 0).astype(int)
        self._gradInput = np.multiply(_gradOutput, mask)
        return self._gradInput
        
    def parameters(self):
        """
        No trainable parametersm, return None
        """
        return None, None

In [6]:
# Optional
class Logistic(Module):
    """
    Logistic activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
        return
    
    def forward(self, _input):
        #YOUR CODE HERE
        self._output = 1.0/(np.exp(-_input) + 1)
        return self._output
    
    def backward(self, _input, _gradOutput):
        #YOUR CODE HERE
        sigmoid = 1.0/(np.exp(-_input) + 1)
        dsigmoid = np.multiply((1 - sigmoid), sigmoid)
        self._gradInput = np.multiply(dsigmoid, _gradOutput)
        return self._gradInput
        
    def parameters(self):
        return None, None

In [7]:
# Optional
class Dropout(Module):
    """
    A dropout layer
    """
    def __init__(self, p = 0.5):
        Module.__init__(self)
        self.p = p #self.p is the drop rate, if self.p is 0, then it's a identity layer
        self.mask = None # A dropout mask for unit
        
    def forward(self, _input):
        self._output = _input
        # YOUR CODE HERE
        # Need to take care of training mode and evaluation mode
        N = _input.shape[0]
        d = _input.shape[1]
        if self.train:
            # Update mask
            self.mask = np.ones([d])
            for i in range(d):
                if random.random < self.p:
                    self.mask[i] = 0
            self.mask = self.mask.reshape([1, d]).repeat(N, axis=0)
            self._output = np.multiply(_input, self.mask)
        else:
            mask_eva = (1 - self.p) * np.ones([N, d])
            self._output = np.multiply(mask_eva, _input)
        return self._output
    
    def backward(self, _input, _gradOutput):
        self._gradInput = _gradOutput
        #YOUR CODE HERE
        self._gradInput = np.multiply(_gradOutput, self.mask)
#         self._gradInput = np.multiply(self._gradInput, _input)
        
        return self._gradInput
    
    def parameters(self):
        """
        No trainable parameters.
        """
        return None, None

In [8]:
class SoftMaxLoss(object):
    def __init__(self):
        return
        
    def forward(self, _input, _label):
        """
        Softmax and cross entropy loss layer. Should return a scalar, since it's a
        loss. (It's almost identical to what in hw2)

        _input: N x C
        _labels: N x C, one-hot

        Returns: loss (scalar)
        """
        #YOUR CODE HERE
        N = _input.shape[0]
        C = _input.shape[1]
        prob = np.exp(_input - _input.max(axis=1).reshape([N, 1]).repeat(C, axis=1))        
        prob = np.divide(prob, np.sum(prob, axis=1).reshape([N, 1]).repeat(C, axis=1))
        logprob = np.log(prob)
        loss = -np.multiply(_label, logprob)
        loss = np.sum(loss, axis=1)
        self._output = np.mean(loss)

        
        return self._output
    
    def backward(self, _input, _label):
        #YOUR CODE HERE
        N = _input.shape[0]
        C = _input.shape[1]
        prob = np.exp(_input - _input.max(axis=1).reshape([N, 1]).repeat(C, axis=1))
        prob = np.divide(prob, np.sum(prob, axis=1).reshape([N, 1]).repeat(C, axis=1))
        self._gradInput = (-_label + prob)*(1.0/N)

        
        return self._gradInput

In [9]:
# Test softmaxloss, the relative error should be small enough
def test_sm():
    crit = SoftMaxLoss()
    gt = np.zeros((3, 5))
    gt[np.arange(3), np.array([1,2,3])] = 1
    x = np.random.random((3, 5))
    def test_f(x):
        return crit.forward(x, gt)
    

    gradInput = crit.backward(x, gt)
    gradInput_num = numeric_gradient(test_f, x, 1, 1e-6)
#     print(gradInput)
#     print(gradInput_num)
    print(relative_error(gradInput, gradInput_num, 1e-8))
    
test_sm()


1.92869894135e-09


In [10]:
# Test modules, all the relative errors should be small enough
def test_module(model):

    # Original: model.evaluate()
    model.training()

    crit = TestCriterion()
    gt = np.random.random((3,10))
    x = np.random.random((3,10))
    def test_f(x):
        return crit.forward(model.forward(x), gt)

    gradInput = model.backward(x, crit.backward(model.forward(x), gt))
    gradInput_num = numeric_gradient(test_f, x, 1, 1e-6)
    print(relative_error(gradInput, gradInput_num, 1e-8))

# Test fully connected
model = FullyConnected(10, 10)
test_module(model)

# Test ReLU
model = ReLU()
test_module(model)

# Test Sigmoid
model = Logistic()
test_module(model)

# Test Dropout
model = Dropout()
test_module(model)
# You can only test dropout in evaluation mode.

# Test Sequential
model = Sequential()
model.add(FullyConnected(10, 10))
model.add(ReLU())
model.add(Dropout())
model.add(FullyConnected(10, 10))
model.add(Logistic())
test_module(model)

3.75534313575e-09
7.36022825478e-10
4.29748384308e-09
7.36022825478e-10
4.13861241453e-08


In [11]:
# Test gradient descent, the loss should be lower and lower
trainX = np.random.random((10,5))

model = Sequential()
model.add(FullyConnected(5, 3))
model.add(Logistic())
model.add(Dropout())
model.add(FullyConnected(3, 1))

crit = TestCriterion()

it = 0
state = None
while True:
    output = model.forward(trainX)
    loss = crit.forward(output, None)
    if it % 100 == 0:
        print(loss)
    doutput = crit.backward(output, None)
    model.backward(trainX, doutput)
    params, gradParams = model.parameters()
#     print("params:{}".format(params))
#     print("grad params: {}".format(gradParams))
    sgd(params, gradParams, 0.01, 0.8) # weight decay 0.8
    if it > 1000:
        break
    it += 1
    

0.0348104871673
0.00848512804721
0.00911629043408
0.00889004964642
0.00883188745352
0.008819959192
0.00881760478166
0.00881714367266
0.00881705353315
0.00881703592436
0.00881703248605


Now we start to work on real data.

In [12]:
import MNIST_utils
data_fn = "CLEAN_MNIST_SUBSETS.h5"

# We only consider large set this time
print("Load large trainset.")
Xlarge,Ylarge = MNIST_utils.load_data(data_fn, "large_train")
print(Xlarge.shape)
print(Ylarge.shape)

print("Load valset.")
Xval,Yval = MNIST_utils.load_data(data_fn, "val")
print(Xval.shape)
print(Yval.shape)

Load large trainset.
(8000, 576)
(8000, 10)
Load valset.
(2000, 576)
(2000, 10)


In [13]:
def predict(X, model):
    """
    Evaluate the soft predictions of the model.
    Input:
    X : N x d array (no unit terms)
    model : a multi-layer perceptron
    Output:
    yhat : N x C array
        yhat[n][:] contains the score over C classes for X[n][:]
    """
    return model.forward(X)

def error_rate(X, Y, model):
    """
    Compute error rate (between 0 and 1) for the model
    """
    model.evaluate()
    res = 1 - (model.forward(X).argmax(-1) == Y.argmax(-1)).mean()
    model.training()
    return res

from copy import deepcopy

def runTrainVal(X,Y,model,Xval,Yval,trainopt):
    """
    Run the train + evaluation on a given train/val partition
    trainopt: various (hyper)parameters of the training procedure
    During training, choose the model with the lowest validation error. (early stopping)
    """
    
    eta = trainopt['eta']
    
    N = X.shape[0] # number of data points in X
    
    # Save the model with lowest validation error
    minValError = np.inf
    saved_model = None
    
    shuffled_idx = np.random.permutation(N)
    start_idx = 0
    
#     fo = open('data/sgd_multidp.csv', 'wb')
#     writer = csv.writer(fo, dialect='excel')
#     writer.writerow(['iteration', 'loss', 'trainError', 'valError'])
    
    for iteration in range(trainopt['maxiter']):
        if iteration % int(trainopt['eta_frac'] * trainopt['maxiter']) == 0:
            eta *= trainopt['etadrop']
        # form the next mini-batch
        stop_idx = min(start_idx + trainopt['batch_size'], N)
        batch_idx = range(N)[int(start_idx):int(stop_idx)]
        bX = X[shuffled_idx[batch_idx],:]
        bY = Y[shuffled_idx[batch_idx],:]

        score = model.forward(bX)
        loss = crit.forward(score, bY)
        # print(loss)
        dscore = crit.backward(score, bY)
        model.backward(bX, dscore)
        
        # Update the data using 
        params, gradParams = model.parameters()
        # sgd
#         sgd(params, gradParams, eta, weight_decay = trainopt['lambda'])
        # sgdm
#         sgdm(params, gradParams, eta, alpha=0.5, weight_decay = trainopt['lambda'])
        # sgdmom
        sgdmom(params, gradParams, eta, alpha = 0.9, weight_decay = trainopt['lambda'])
    
        start_idx = stop_idx % N
        
        
        if (iteration % trainopt['display_iter']) == 0:
            #compute train and val error; multiply by 100 for readability (make it percentage points)
            trainError = 100 * error_rate(X, Y, model)
            valError = 100 * error_rate(Xval, Yval, model)
            print('{:8} batch loss: {:.3f} train error: {:.3f} val error: {:.3f}'.format(iteration, loss, trainError, valError))
#             writer.writerow([iteration, loss, trainError, valError])
            if valError < minValError:
                saved_model = deepcopy(model)
                minValError = valError
#     fo.close()
        
    return saved_model, minValError, trainError

In [14]:
def build_model(input_size, hidden_size, output_size, activation_func = 'ReLU', dropout = 0):
    """
    Build the model:
    input_size: the dimension of input data
    hidden_size: the dimension of hidden vector
    output_size: the output size of final layer.
    activation_func: ReLU, Logistic, Tanh, etc. (Need to be implemented by yourself)
    dropout: the dropout rate: if dropout == 0, this is equivalent to no dropout
    """
    #YOUR CODE HERE
    h1 = hidden_size
    h2 = int(hidden_size/2)
    model = Sequential()
    model.add(FullyConnected(input_size, h1))
    model.add(ReLU())
    model.add(Dropout(dropout))
    model.add(FullyConnected(h1, h2))
    model.add(ReLU())
    model.add(Dropout(dropout))
#     for i in range(0):
#         model.add(FullyConnected(hidden_size, hidden_size))
#         if activation_func == 'ReLU':
#             model.add(ReLU())
    model.add(FullyConnected(h2, output_size))
    
    return model
    

In [15]:
# -- training options
trainopt = {
    'eta': 0.01,   # initial learning rate
    'maxiter': 20000,   # max number of iterations (updates) of SGD
    'display_iter': 500,  # display batch loss every display_iter updates
    'batch_size': 100,  
    'etadrop': .5, # when dropping eta, multiply it by this number (e.g., .5 means halve it)
    'eta_frac': .25  #
}

NFEATURES = Xlarge.shape[1]

# we will maintain a record of models trained for different values of lambda
# these will be indexed directly by lambda value itself
trained_models = dict()

# set the (initial?) set of lambda values to explore
lambdas = np.array([0, 0.001, 0.01, 0.1]) # 0, 0.001, 0.01, 0.1
hidden_sizes = np.array([400]) # 50, 100, 200

    
for lambda_ in lambdas:
    for hidden_size_ in hidden_sizes:
        trainopt['lambda'] = lambda_
        model = build_model(NFEATURES, hidden_size_, 10, dropout = 0)
        crit = SoftMaxLoss()
        # -- model trained on large train set
        trained_model,valErr,trainErr = runTrainVal(Xlarge, Ylarge, model, Xval, Yval, trainopt)
        trained_models[(lambda_, hidden_size_)] = {'model': trained_model, "val_err": valErr, "train_err": trainErr }
        print('train set model: -> lambda= %.4f, train error: %.2f, val error: %.2f' % (lambda_, trainErr, valErr))
    
best_trained_lambda = 0.
best_trained_model = None
best_trained_val_err = 100.
for (lambda_, hidden_size_), results in trained_models.items():
    print('lambda= %.4f, hidden size: %5d, val error: %.2f' %(lambda_, hidden_size_, results['val_err']))
    if results['val_err'] < best_trained_val_err:
        best_trained_val_err = results['val_err']
        best_trained_model = results['model']
        best_trained_lambda = lambda_

print("Best train model val err:", best_trained_val_err)
print("Best train model lambda:", best_trained_lambda)
print("Analysis is in pdf")

       0 batch loss: 2.293 train error: 89.487 val error: 88.800
     500 batch loss: 1.588 train error: 29.663 val error: 27.650
    1000 batch loss: 0.684 train error: 16.300 val error: 13.400
    1500 batch loss: 0.497 train error: 12.162 val error: 10.250
    2000 batch loss: 0.408 train error: 10.525 val error: 9.500
    2500 batch loss: 0.333 train error: 9.700 val error: 8.950
    3000 batch loss: 0.248 train error: 8.713 val error: 8.550
    3500 batch loss: 0.317 train error: 8.125 val error: 8.050
    4000 batch loss: 0.282 train error: 7.588 val error: 7.750
    4500 batch loss: 0.241 train error: 7.288 val error: 7.200
    5000 batch loss: 0.177 train error: 6.988 val error: 7.100
    5500 batch loss: 0.263 train error: 6.712 val error: 7.000
    6000 batch loss: 0.230 train error: 6.600 val error: 7.050
    6500 batch loss: 0.206 train error: 6.425 val error: 6.900
    7000 batch loss: 0.156 train error: 6.237 val error: 6.650
    7500 batch loss: 0.241 train error: 6.125 

In [16]:
#Generate a Kaggle submission file using `model`
kaggleX = MNIST_utils.load_data(data_fn, 'kaggle')
kaggleYhat = predict(kaggleX, best_trained_model).argmax(-1)
save_submission('submission-mnist.csv', kaggleYhat)

('Saved:', 'submission-mnist.csv')
